## Import Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

## Load data

In [ ]:
data = pd.read_csv("myfile.csv")

## Preprocessing

#### Normalize

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
data_scaled = scaler.fit_transform(data)

#### Convert the data to PyTorch tensors

In [ ]:
data_tensor = torch.Tensor(data_scaled)

## Model Initialization

In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)
    
    def forward(self, input):
        lstm_out, _ = self.lstm(input.view(len(input), 1, -1))
        output = self.linear(lstm_out.view(len(input), -1))
        return output[-1]

In [ ]:
input_size = len(data.columns) - 1  # Number of input features (excluding the target)
hidden_size = 32  # Number of hidden units in the LSTM layer
output_size = 1  # Number of output predictions (1 for predicting stock price)

## Training

#### Loss 

In [ ]:
criterion = nn.MSELoss()

#### Optimizer

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)

#### Split Data into train and test (below number is just an example)

In [ ]:
train_data = data_tensor[:200]
test_data = data_tensor[200:]

#### Training loop

In [ ]:
num_epochs = 100
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    train_outputs = model(train_data[:-1])
    train_labels = train_data[1:, 0]  # Assuming the target variable is in the first column
    loss = criterion(train_outputs, train_labels)
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()
    
    # Print the loss for monitoring
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

## Evaluate the model on the test data

In [ ]:
model.eval()
test_outputs = model(test_data[:-1])
test_labels = test_data[1:, 0]  # Assuming the target variable is in the first column
test_loss = criterion(test_outputs, test_labels)

# Denormalize the predictions and ground truth values
test_outputs = scaler.inverse_transform(test_outputs.detach().numpy())
test_labels = scaler.inverse_transform(test_labels.detach().numpy())

# Print the test loss and the predicted stock prices
print(f'Test Loss: {test_loss.item()}')
print('Predicted Prices:')
for pred in test_outputs:
    print(pred)